# Getting Google Trends data

In [1]:
!pip install pytrends

In [ ]:
from pytrends.request import TrendReq
import pandas as pd

# List of terms to search for
termos = pd.read_csv("data/softwares_names.csv")
termos = termos["softwares_names"].tolist()

# Sppliting the list in 5 groups
grupos_de_termos = [termos[i:i+1] for i in range(0, len(termos), 1)]

# Creating an instance of TrendReq
pytrends = TrendReq(hl='pt-BR', tz=360)  # Defina a linguagem (hl) e o fuso horário (tz)
resultados = pd.DataFrame()

# For each term group
for grupo in grupos_de_termos:
    # Setting the search parameters
    pytrends.build_payload([grupo], cat=5, timeframe='all', geo='', gprop='')
    # Getting data
    dados_novos = pytrends.interest_over_time()
    resultados = pd.concat([resultados, dados_novos], axis=0, join='outer')

In [2]:
agregate = resultados.groupby("date").sum()

In [3]:
agregate

,pandas,isPartial
date,,
2004-01-01,2,0
2004-02-01,1,0
2004-03-01,5,0
2004-04-01,0,0
2004-05-01,2,0
...,...,...
2023-06-01,67,0
2023-07-01,62,0
2023-08-01,67,0


In [4]:
agregate = agregate.reset_index()
agregate = agregate.melt(id_vars=["date"], 
        var_name="project", 
        value_name="relative_interest")

In [5]:
termos = pd.read_csv("data/software_names.csv")
termos

,id,softwares_names
0,1,pandas


In [6]:
merge = pd.merge(agregate, termos, left_on="project", right_on="softwares_names", how="left")

In [7]:
merge = merge.groupby(["date", "id"]).agg({"project": "first", "relative_interest": "sum"})

In [8]:
merge = merge.reset_index()
merge.drop(columns={"id"}, inplace = True)
merge

,date,project,relative_interest
0,2004-01-01,pandas,2
1,2004-02-01,pandas,1
2,2004-03-01,pandas,5
3,2004-04-01,pandas,0
4,2004-05-01,pandas,2
...,...,...,...
233,2023-06-01,pandas,67
234,2023-07-01,pandas,62
235,2023-08-01,pandas,67
236,2023-09-01,pandas,68


In [37]:
merge.to_csv("data/trends.csv")